In [ ]:
import toml
import pygsheets
from tqdm.auto import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import benchlingapi
import sqlalchemy
from sqlalchemy import select
import xmltodict
from lxml import etree

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api.benchling as bapi
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.io as cio
import paulssonlab.cloning.sequence as sequence
import paulssonlab.api.geneious as geneious
import paulssonlab.api.geneious_db as g

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Config

In [ ]:
lib_parts = reg[("LIB", "parts")]
olib_oligos = reg[("oLIB", "oligos")]
plib_maps = reg[("pLIB", "maps")]
part_types = reg[("LIB", "parts", "Part types")]

# Test

## XML

In [ ]:
seq = plib_maps["pLIB112"]

In [ ]:
from xmldiff.main import diff_texts

In [ ]:
output = etree.tostring(geneious.dump_geneious(seq4), pretty_print=True)

In [ ]:
x = etree.fromstring(doc4)

In [ ]:
geneious.load_geneious(x)

In [ ]:
seq4.annotations

In [ ]:
seq4.annotations["references"][0].__class__

In [ ]:
geneious.loads_geneious(doc4).annotations

## Round-trip: seq->geneious-seq

## Round-trip: geneious->seq->geneious

## DB

In [ ]:
db_url = sqlalchemy.engine.URL.create(**config["geneious_test"])
engine = sqlalchemy.create_engine(db_url)
conn = engine.connect()

In [ ]:
Session = sqlalchemy.orm.sessionmaker(bind=engine)
session = Session()

## DB test

In [ ]:
session.query(g.Folder).where(g.Folder.visible == True).count()

In [ ]:
session.query(g.Folder).count()

In [ ]:
root = session.query(g.Folder).all()[0]

In [ ]:
root.children

## Old

In [ ]:
q = select(g.Folder).where(g.Folder.visible == True)
res = conn.execute(q)

In [ ]:
res.all()

In [ ]:
conn.execute(select(g.NextTableId)).all()

In [ ]:
for name, t in metadata.tables.items():
    print(name)
    q = select([t]).limit(20)
    rows = conn.execute(q).all()
    for row in rows:
        print(row)
    print()

In [ ]:
q = select(g.AnnotatedDocument)
rows = conn.execute(q).all()

In [ ]:
doc0 = rows[0]["plugin_document_xml"]

In [ ]:
doc4 = rows[-1]["plugin_document_xml"]

In [ ]:
import os

In [ ]:
seq4 = cio.read_file(os.path.expanduser("/Users/jacob/Downloads/_plasmids/genbank2.gb"))

In [ ]:
seq4

In [ ]:
print(doc4)

In [ ]:
doc2 = rows[1]["plugin_document_xml"]

In [ ]:
rows[5]["xml_element"]

In [ ]:
docs = [xmltodict.parse(row["document_xml"], dict_constructor=dict) for row in rows]

In [ ]:
BAD_CLASSES = [
    # "com.biomatters.geneious.publicapi.implementations.sequence.OligoSequenceDocument",
    "com.biomatters.geneious.publicapi.documents.OperationRecordDocument",
]
good_docs = [d for d in docs if d["document"]["@class"] not in BAD_CLASSES]

In [ ]:
set(d["document"]["@class"] for d in good_docs)

In [ ]:
rows[2]["document_xml"]

In [ ]:
x = xmltodict.parse(rows[2]["plugin_document_xml"], dict_constructor=dict)

In [ ]:
x